In [1]:
!pip install tensorflow==2.16.0rc0
!PYTHONHASHSEED=0
!pip install patchify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 804.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 40.1 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    

In [2]:
# Import other modules
from matplotlib import pyplot as plt
import zipfile
from shutil import copyfile
from time import time
import numpy as np
import pandas as pd
import random as python_random
import os
import shutil
import glob
from patchify import patchify, unpatchify
import random

# Import TensorFlow/Keras
import tensorflow as tf
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, Conv3DTranspose, MaxPooling3D, concatenate, Dropout, Activation, BatchNormalization, GroupNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow.keras.backend as K


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Percorso del file ZIP su Google Drive
drive_zip_path = '/content/drive/MyDrive/DL_PROJECT/Data/Test_128.zip'
local_extract_path = '/content/Test_Data'
os.makedirs(local_extract_path, exist_ok=True)

# Decomprimere il file ZIP nella directory locale
with zipfile.ZipFile(drive_zip_path, 'r') as zip_ref:
    zip_ref.extractall(local_extract_path)

print("Decompressione completata.")

Decompressione completata.


In [4]:
import numpy as np  # Import the numpy library for array manipulation

# Function to load images from a given directory and list of image file names
def load_img(img_dir, img_list):
    images = []  # Initialize an empty list to store the images
    for image_name in img_list:  # Loop through each image name in the list
        if image_name.split('.')[-1] == 'npy':  # Check if the file is a .npy file
            volume = np.load(img_dir + image_name)  # Load the .npy file as a numpy array
            # Extract 128 images along the third axis (z-axis) from each volume
            for j in range(volume.shape[2]):
                images.append(volume[:, :, j, :])  # Append each slice to the images list
    images = np.array(images)  # Convert the list of images to a numpy array
    return images  # Return the numpy array of images

# Generator function to load and yield image and mask batches
def image_loader(img_dir, img_list, mask_dir, mask_list, batch_size=256):
    assert batch_size == 256, "Batch size must be 256 to match the number of slices per two volumes."  # Ensure batch size is 256

    L = len(img_list)  # Get the length of the image list

    while True:  # Infinite loop to keep yielding batches
        for i in range(0, L, 2):  # Iterate through the image list in steps of 2
            if i + 1 < L:  # Check if there are at least two more volumes to load
                img = load_img(img_dir, [img_list[i], img_list[i+1]])  # Load images from two volumes
                mask = load_img(mask_dir, [mask_list[i], mask_list[i+1]])  # Load corresponding masks

                # Combine images and masks into a list of tuples and shuffle them
                combined = list(zip(img, mask))
                np.random.shuffle(combined)
                img[:], mask[:] = zip(*combined)  # Unzip the shuffled list back into images and masks

                yield (img, mask)  # Yield the shuffled images and masks as a batch
            else:
                # Handle the case where there is an odd number of volumes
                img = load_img(img_dir, [img_list[i]])  # Load images from the last volume
                mask = load_img(mask_dir, [mask_list[i]])  # Load corresponding masks

                # Combine images and masks into a list of tuples and shuffle them
                combined = list(zip(img, mask))
                np.random.shuffle(combined)
                img[:], mask[:] = zip(*combined)  # Unzip the shuffled list back into images and masks

                yield (img, mask)  # Yield the shuffled images and masks as a batch

In [5]:
import tensorflow.keras.backend as K

def DiceCoefficient(y_true, y_pred, smooth = 1e-6):

    y_pred = tf.keras.activations.softmax(y_pred, axis = -1)

    # Cast to float32 datatype
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')

    # Flatten label and prediction tensors
    inputs = K.flatten(y_pred)
    targets = K.flatten(y_true)

    intersection = K.sum(inputs * targets)
    dice = (2 * intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    return dice

In [6]:
model = keras.saving.load_model('/content/drive/MyDrive/DL_PROJECT/2D_FINAL/2D_Unet_final_2.keras', compile = True , custom_objects={'DiceCoefficient' : DiceCoefficient})

In [7]:
test_img_dir = '/content/Test_Data/X_test/'
test_mask_dir = '/content/Test_Data/Y_test/'

test_img_list = sorted(os.listdir(test_img_dir))
test_mask_list = sorted(os.listdir(test_mask_dir))

In [8]:
batch_size = 256

test_img_datagen = image_loader(test_img_dir, test_img_list, test_mask_dir, test_mask_list, batch_size)

steps_per_epoch = len(test_img_list)//2
print(steps_per_epoch)

36


In [9]:
evaluation = model.evaluate(
    test_img_datagen,
    verbose = 1,
    steps = steps_per_epoch,
    callbacks = None,
    return_dict = True
)

36/36 ━━━━━━━━━━━━━━━━━━━━ 1468s 41s/step - accuracy: 0.9087 - dice_coefficient: 0.4133 - loss: 0.2906 - one_hot_io_u: 0.9149 - one_hot_io_u_1: 0.2796 - one_hot_io_u_2: 0.3091 - one_hot_io_u_3: 0.2961 - one_hot_mean_io_u: 0.4499


In [13]:
evaluation

{'accuracy': 0.929600179195404,
 'dice_coefficient': 0.42059606313705444,
 'loss': 0.30292290449142456,
 'one_hot_io_u': 0.9361362457275391,
 'one_hot_io_u_1': 0.32653361558914185,
 'one_hot_io_u_2': 0.24520477652549744,
 'one_hot_io_u_3': 0.33646467328071594,
 'one_hot_mean_io_u': 0.46108490228652954}